In [1]:
# train svm model

In [2]:
import sys
import pandas as pd
sys.path.append("../src")


from create_folds import create_folds
from train import run
from model_dispatcher import models
from utilities.categorical_encoders import Encoder
from utilities.feature_engineering import *
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import config

In [3]:
train_df = pd.read_csv("../data/fraudTrain.csv")

In [4]:
train_df = create_folds(train_df, True)

In [5]:
def process(train_df):
    # Encode categorical features using label encoder
    categorical_features = ['merchant','category', 'gender', 'city', 'state']
    encoder = Encoder(encoder="label_encoder", columns_names=categorical_features)
    train_df = encoder.fit_transform(train_df)

    # Extact date features from the transaction date
    train_df["trans_date_trans_time"] = pd.to_datetime(train_df["trans_date_trans_time"], format="%Y-%m-%d %H:%M:%S")
    train_df = extract_date_features(train_df, "trans_date_trans_time")

    # Calculate age of the user
    train_df["dob"] = pd.to_datetime(train_df["dob"], format="%Y-%m-%d")
    train_df["age"]  = train_df["trans_date_trans_time"]-train_df["dob"]
    train_df["age"] = round(train_df["age"].dt.days/365)

    # Drop unuseful columns
    col_to_drop = config.COL_TO_DROP
    print(f"__COL_TO_DROP__: {col_to_drop}")
    train_df = train_df.drop(col_to_drop, axis=1)

    # Oversamping
    sm = SMOTE()
    train_df[[col for col in train_df.columns if col not in [config.TARGET]]], train_df["is_fraud"] = sm.fit_resample(train_df.drop("is_fraud", axis=1), train_df["is_fraud"])
    
train_df = process(train_df)

../src\utilities\feature_engineering.py:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df["week"] = df[date_col].dt.week


__COL_TO_DROP__: ['Unnamed: 0', 'cc_num', 'first', 'last', 'street', 'zip', 'trans_date_trans_time', 'dob', 'trans_num', 'unix_time', 'job', 'merch_long', 'merch_lat']


In [6]:

# train the model using the goldout strategy for validation
model = "random_forest_classifier"
error="f1_score"
clfs = [run(train_df, fold, model, error) for fold in train_df.kfold.unique()]

AttributeError: 'NoneType' object has no attribute 'kfold'